In [1]:
import pandas as pd
import numpy as np 
import nltk
import spacy

In [14]:
from spacy import displacy
from tqdm import tqdm

In [3]:
df0 = pd.read_csv("medium_articles.csv")
df0

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."
...,...,...,...,...,...,...
192363,Why do you need a cleaning service?,What could be more important than having a tid...,https://medium.com/@ozneedcleaningau/why-do-yo...,[],2021-11-16 08:17:08.950000+00:00,"['Cleaning', 'Cleaning Services', 'Cleaning Co..."
192364,Daily cleaning and maintenance of bedding,Daily cleaning and maintenance of bedding\n\nW...,https://medium.com/@a198blwt/daily-cleaning-an...,[],2021-11-16 05:27:05.359000+00:00,"['Bedding', 'Cleaning', 'Maintain']"
192365,Beneficial Advice on Bond Cleaning!,The most important chore at the end is bond cl...,https://medium.com/@princegohil/beneficial-adv...,['Prince Shrawan'],2021-11-26 08:20:27.660000+00:00,"['Cleaning', 'End Of Lease Cleaning', 'Cleaners']"
192366,How I Learned Romanian in 37 Easy Steps,How I Learned Romanian in 37 Easy Steps\n\nHey...,https://medium.com/@lifeinromania/how-i-learne...,['Sam Ursu'],2017-11-27 08:09:19.025000+00:00,"['Romania', 'Language Learning', 'Storyofmylife']"


In [4]:
# Taking the "text column" for preprocessing
df = df0["text"]
df["articles"] = df.iloc[:]
df.info()

<class 'pandas.core.series.Series'>
Index: 192369 entries, 0 to articles
Series name: text
Non-Null Count   Dtype 
--------------   ----- 
192369 non-null  object
dtypes: object(1)
memory usage: 2.9+ MB


### Text preprocessing

#### Removed the stopwords with spacy

In [5]:
import spacy
NER = spacy.load('en_core_web_sm')
stopwords = NER.Defaults.stop_words

print(len(stopwords))
print(stopwords)


C:\Users\Acer\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


326
{'been', 'then', 'may', 'themselves', 'into', 'seeming', 'used', 'either', 'namely', 'never', 'towards', 'between', 'regarding', 'toward', 'out', 'who', 'somehow', 'did', 'really', 'could', 'alone', 'around', 'top', 'beside', 'one', 'though', 'sometimes', 'own', 'make', 'neither', 'side', 'same', 'do', 'have', 'here', 'this', 'two', 'in', 'latterly', 'perhaps', 'nine', 'else', 'noone', 'somewhere', 'below', 'behind', 'already', 'mine', 'everywhere', 'three', 'yourselves', 'most', 'least', 'throughout', 'but', 'nowhere', 'empty', 'while', 'full', 'together', "'ll", 'another', 'whoever', 'be', 'onto', 'thereupon', 'to', 'still', 'would', 'our', 'otherwise', 'i', 'various', 'mostly', 'n’t', 'because', 'once', 'had', 'again', 'for', 'whereby', 'due', 'some', 'sometime', 'thereby', 'whither', 'and', 'cannot', 'such', 'itself', 'will', 'your', 'where', 'always', 'done', 'put', 'must', 'ours', 'too', 'thus', 'five', 'their', 'might', 'anyway', 'can', 'at', 'hereafter', 'now', 'say', 'the'

In [6]:
# for every article(row) in df["articles"], split into words & lowercase
# append into a list removing the stopwards
article_without_sw = [[j for j in i.split() if j.lower() not in stopwords] for i in tqdm(df["articles"])]


100%|████████████████████████████████████████████████████████████████████████| 192368/192368 [01:27<00:00, 2203.41it/s]


In [11]:
article_join = [' '.join(i) for i in tqdm(article_without_sw)]

100%|███████████████████████████████████████████████████████████████████████| 192368/192368 [00:08<00:00, 21570.95it/s]


#### Extracting the Named Entities ('GPE','PERSON',"ORG") from each article(row)

In [364]:
entities = []
labels= []
for i in tqdm(article_join): 
    entity = [ent.text for ent in NER(i).ents if ent.label_ in ('GPE','PERSON',"ORG")]
    label = [ent.label_ for ent in NER(i).ents if ent.label_ in ('GPE','PERSON',"ORG")]
    entities.append(entity)
    labels.append(label)


  0%|                                                                           | 199/192368 [00:31<7:51:19,  6.80it/s]


  0%|▏                                                                          | 396/192368 [01:06<6:50:14,  7.80it/s]


  0%|▏                                                                         | 582/192368 [01:49<16:05:56,  3.31it/s]


  0%|▎                                                                          | 773/192368 [02:34<9:01:30,  5.90it/s]


  1%|▎                                                                         | 963/192368 [03:19<19:10:58,  2.77it/s]


...

 17%|████████████▏                                                         | 33454/192368 [3:56:19<10:34:31,  4.17it/s]


 17%|████████████▏                                                         | 33637/192368 [3:57:06<16:17:55,  2.71it/s]


 18%|████████████▎                                                         | 33795/192368 [3:57:54<14:36:49,  3.01it/s]


 18%|████████████▎                                                         | 33875/192368 [3:58:15<18:34:43,  2.37it/s]


KeyboardInterrupt: 

In [366]:
len(entities)

33875

### Removing duplicates from 1st list of NER (entities)

In [377]:
ent_li33875 = []
for i in tqdm(entities):
    k = list(set(i))
    ent_li33875.append(k)
    

100%|█████████████████████████████████████████████████████████████████████████| 33875/33875 [00:00<00:00, 59302.73it/s]


In [379]:
len(ent_li33875)

33875

### NER for reamining data from[33875:]

In [386]:
entities_1 = []
for i in tqdm(article_join[33876:]): 
    entity = list(set([ent.text for ent in NER(i).ents if ent.label_ in ('GPE','PERSON',"ORG")]))
    entities_1.append(entity)

100%|███████████████████████████████████████████████████████████████████████| 158492/158492 [19:00:24<00:00,  2.32it/s]


In [388]:
# joining two lists entities + entities_1
articles_ner = entities + entities_1
len(articles_ner)

192367

### Making the list to dataframe

In [396]:
df_ner = pd.DataFrame(articles_ner)
df_ner    

,0,1,2,3,4,5,6,7,8,9,...,2208,2209,2210,2211,2212,2213,2214,2215,2216,2217
0,Josh Riemer Unsplash,Ryan,Juliette,Marie,Meredith,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Pexels,Zoom,ACE2,New England Journal Medicine,Psychiatric Disorders Photo Jonathan Rados,COVID-19,Denmark,San Francisco,COVID-19 Long-Haulers Thousands,COVID-19,...,None,None,None,None,None,None,None,None,None,None
2,Ann-Sophie Barwich,Johannes Frasnelli,University Quebec Trois-Rivières,Frasnelli’s,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Phineas Gage,Cherry,Phineas Gage,Phineas Gage,Vermont,Cherry,Phineas Gage,Phineas Gage,Phineas Gage,Cherry,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192362,Cleaners,North Shore,Sydney North Shore,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
192363,Hemp,Cotton,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
192364,DIY,Bond Cleaning Adelaide,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
192365,Grin,Bucharest,Transylvania,SEH-RA,Beetle Bailey,Righteous Scolding,Nada,I’m Romanian!,LACE,KA-CHING!,...,None,None,None,None,None,None,None,None,None,None


In [397]:
df_ner.to_csv('NER_entities.csv')

In [398]:
df_ner_final = pd.DataFrame({"articles_ner" : articles_ner})
df_ner_final

,articles_ner
0,"[Josh Riemer Unsplash, Ryan, Juliette, Marie, ..."
1,"[Pexels, Zoom, ACE2, New England Journal Medic..."
2,"[Ann-Sophie Barwich, Johannes Frasnelli, Unive..."
3,[]
4,"[Phineas Gage, Cherry, Phineas Gage, Phineas G..."
...,...
192362,"[Cleaners, North Shore, Sydney North Shore]"
192363,"[Hemp, Cotton]"
192364,"[DIY, Bond Cleaning Adelaide]"
192365,"[Grin, Bucharest, Transylvania, SEH-RA, Beetle..."


In [399]:
df_ner_final.to_csv('NER_entities_final.csv')